# Electricity_Demand

**Source:** *https://www.ree.es/en/apidatos*

In [1]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import itertools
from functools import reduce
import operator

pd.options.display.max_columns=None

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Obtenemos información de la estación seleccionada durante el periodo deseado
API_stream_url = f'https://apidatos.ree.es/en/datos/demanda/evolucion?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=day'
API_stream_url

'https://apidatos.ree.es/en/datos/demanda/evolucion?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=day'

In [3]:
# Importamos la librería requests
import requests

# Realizamos el request
r = requests.get(API_stream_url)
r_json = r.json()
r

<Response [200]>

In [4]:
# Obtenemos los datos necesarios
col_2_s = lambda col: pd.DataFrame(col['attributes']['values']).set_index('datetime')['value']

def get_from_dict(data_dict, map_list):
    return reduce(operator.getitem, map_list, data_dict)

def json_2_nested_lists(obj_in, route):
    if len(route) == 0:
        yield obj_in
        
    elif isinstance(obj_in, list):
        for item in obj_in:
            obj_out = json_2_nested_lists(item, route)
            yield from obj_out
            
    else:
        obj_out = get_from_dict(obj_in, route[0])
        yield from json_2_nested_lists(obj_out, route[1:])

def json_2_df(r_json, JSON_route):
    nested_lists = json_2_nested_lists(r_json, JSON_route)
    cols_flatlist = list(itertools.chain(*nested_lists))

    df = pd.DataFrame()

    for col in cols_flatlist:
        s_data = col_2_s(col)
        s_data.name = col['type']

        df[s_data.name] = s_data

    df.index = pd.to_datetime(df.index)
    
    return df

JSON_route = [
    ['included'],
]
   
df = json_2_df(r_json, JSON_route)

df.head()

,Demand
datetime,
2022-01-01 00:00:00+01:00,535144.786
2022-01-02 00:00:00+01:00,578855.186
2022-01-03 00:00:00+01:00,691970.774
2022-01-04 00:00:00+01:00,718047.874
2022-01-05 00:00:00+01:00,715070.234


In [5]:
# Importamos la librería datetime
import datetime as dt

# Dotamos a la fecha del formato deseado
def limpiar_fecha(fecha):
    nueva_fecha = dt.datetime.strptime(str(fecha), '%Y-%m-%d %H:%M:%S%z').replace(tzinfo=None)
    return nueva_fecha

dataset = df.reset_index(drop=False)
dataset['Date'] = dataset['datetime'].apply(lambda x: limpiar_fecha(x))
dataset = dataset.drop('datetime', axis=1)
dataset

,Demand,Date
0,535144.786,2022-01-01
1,578855.186,2022-01-02
2,691970.774,2022-01-03
3,718047.874,2022-01-04
4,715070.234,2022-01-05
...,...,...
360,611568.630,2022-12-27
361,629533.183,2022-12-28
362,636823.461,2022-12-29
363,623610.426,2022-12-30
